In [59]:
# 필요한 라이브러리 불러오기
import os
import random
import argparse
from tqdm import tqdm
from datetime import datetime, timezone, timedelta
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [95]:
# 시드 고정
RANDOM_SEED = 10
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

# 경로 설정
ROOT_PATH = 'C:/Users/crid2/'

# DATA_DIR = os.path.join(ROOT_PATH, 'data')
DATA_DIR = 'kepcodata'

In [96]:
df = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))

In [97]:
df.head()

,index,누적전력량,유효전력평균,무효전력평균,주파수,전류평균,상전압평균,선간전압평균,온도,R상유효전력,...,S상전압,S상선간전압,T상유효전력,T상무효전력,T상전류,T상전압,T상선간전압,label_역률평균,label_전류고조파평균,label_전압고조파평균
0,0,2248930.5,28963.0,20237.0,59.854076,45.197918,259.916656,449.916656,47.500,8663.0,...,260.75,452.75,10417.0,7304.0,48.71875,260.75,448.75,정상,경고,주의
1,1,0.0,0.0,0.0,0.000000,101.312500,0.000000,0.000000,24.375,0.0,...,0.00,0.00,0.0,0.0,100.68750,0.00,0.00,경고,정상,정상
2,2,5375707.0,35244.0,19826.0,59.975650,107.385414,125.416664,216.750000,18.750,11988.0,...,125.25,216.25,12236.0,6170.0,109.56250,125.00,216.25,정상,정상,주의
3,3,17781200.0,77056.0,39520.0,59.863000,244.854000,118.083000,205.333000,23.125,25796.0,...,118.75,118.75,24992.0,13704.0,242.18800,118.00,118.00,정상,정상,경고
4,4,10143988.0,0.0,0.0,59.798140,0.000000,133.750000,231.500000,26.875,0.0,...,134.50,231.50,0.0,0.0,0.00000,133.00,230.25,경고,정상,정상


In [98]:
# 학습 데이터의 감정 분류 분포
df[['index','label_전압고조파평균']].groupby('label_전압고조파평균').count().rename(columns={'':'count'})

,index
label_전압고조파평균,
경고,735155
정상,994194
주의,691216


In [87]:
# 데이터로드 파라미터
BATCH_SIZE = 2048
NUM_WORKERS = 0

In [88]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler
import joblib

In [89]:
    class CustomDataset(Dataset) : 
    #             """ 필수 함수 : 
    #             - __init__ : 초기화
    #             - __len__ : 데이터셋(input)의 길이 반환
    #             - __getitem__ : 데이터셋을 인덱스로 불러옴
    #         """

    #         """CustomDataset 클래스 정의

    #         args:
    #             data_dir (str)
    #             mode (str)
    #         """
        def __init__(self, data_dir, mode='train') : 
            self.mode = mode
            self.data_dir = data_dir

            # 테스트 모드 디코딩에 사용할 레이블 딕셔너리 
            self.label_dic = {0:'정상' , 1:'주의' , 2:'경고'}

            print('Loading ' + self.mode + ' dataset..')

            # 데이터 경로가 존재하는지 확인
            if not os.path.isdir(self.data_dir) : 
                print(f'!!!, Cannot find {self.data_dir}....!!!')
                sys.exit()

            # csv파일로드 
            # val모드에서 train 데이터를 가져옵니다..? 

            if self.mode =='val' :
                self.data_path = os.path.join(self.data_dir, 'train.csv')
            else : 
                self.data_path = os.path.join(self.data_dir, f'{mode}.csv')

            df = pd.read_csv(self.data_path)


            # 학습/검증 데이터 구역선택 , 학습 데이터 비율에 따라 TRAIN_RATIO 변경
            TRAIN_RATIO = 0.9
            if self.mode == 'train':
                df = df[:int(len(df)*TRAIN_RATIO)]

            elif self.mode == 'val':
                df = df[int(len(df)*TRAIN_RATIO)+1:].reset_index(drop=True)



            if self.mode != 'test': 
                features = df.iloc[: , 1:-3].copy().fillna(0)
                targets = df.iloc[:  -3:].copy().fillna(0)

            else : 
                features = df.iloc[:, 1:].copy().fillna(0)

            feature_scaler = MinMaxScaler()
            feature_scaler_path = 'feature_scaler.pkl'

            if self.mode == 'train':
                feature_scaler.fit(features)
                joblib.dump(feature_scaler, feature_scaler_path)
            else:
                feature_scaler = joblib.load(feature_scaler_path)

            self.features = feature_scaler.transform(features)




            # target encoding : 
            if self.mode != 'test' : 
                self.targets = targets.replace({
                    '정상' : 0,
                    '주의' : 1,
                    '경고' : 2

                })

        def label_decoder(self, labels) : 
            try : 
                labels = list(mp(lambda x : self.label_dic[x] , labels))
                return labels
            except : 
                assert 'Invalid intent'


        def __len__(self) : 
            return self.features.shape[0]


        # 23개의 피쳐들과 test 모드일 때를 제외하고 target 세가지 를 리턴 

        def __getitem__(self, idx:int) : 

            feature = self.features[idx]  
            feature = torch.tensor(feature, dtype=torch.float)

            if self.mode != 'test' : 
                target_1 = int(self.targets.iloc[idx, 0])
                target_2 = int(self.targets.iloc[idx, 1])
                target_3 = int(self.targets.iloc[idx, 2])

            else : 
                return feature


        
    
            

In [99]:
# DATASET 만들기 

train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train')
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val')

Loading train dataset..
Loading val dataset..


In [100]:
# DATASET 로딩하기
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False, drop_last=False)

In [101]:
# hyper-parameters
LEARNING_RATE = 0.0001
EPOCHS = 1000

NUM_FEATURES = 23
HIDDEN_SIZE = 4096
EARLY_STOPPING_PATIENCE = 20


In [102]:
class CNN1D(nn.Module):
    def __init__(self, num_features, num_targets=3, hidden_size=2048):
        super(CNN1D, self).__init__()
        cha_1 = 256
        cha_2 = 512
        cha_3 = 512

        cha_1_reshape = int(hidden_size/cha_1)
        cha_po_1 = int(cha_1_reshape/2)
        cha_po_2 = int(cha_po_1/2) * cha_3

        self.cha_1 = cha_1
        self.cha_2 = cha_2
        self.cha_3 = cha_3
        self.cha_1_reshape = cha_1_reshape
        self.cha_po_1 = cha_po_1
        self.cha_po_2 = cha_po_2

        # Dense
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

        # CNN 1
        self.batch_norm_c1 = nn.BatchNorm1d(cha_1)
        self.dropout_c1 = nn.Dropout(0.1)
        self.conv1 = nn.utils.weight_norm(nn.Conv1d(cha_1,cha_2, kernel_size = 5, stride = 1, padding=2,  bias=False),dim=None)

        # Avg-Pool
        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = cha_po_1)

        # CNN 2_0
        self.batch_norm_c2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2 = nn.Dropout(0.1)
        self.conv2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

        # CNN 2_1
        self.batch_norm_c2_1 = nn.BatchNorm1d(cha_2)
        self.dropout_c2_1 = nn.Dropout(0.3)
        self.conv2_1 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

        # CNN 2_2
        self.batch_norm_c2_2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2_2 = nn.Dropout(0.2)
        self.conv2_2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_3, kernel_size = 5, stride = 1, padding=2, bias=True),dim=None)

        # Max-Pool
        self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

        # Flatten
        self.flt = nn.Flatten()

        # Dense 1
        self.batch_norm3_1 = nn.BatchNorm1d(cha_po_2)
        self.dropout3_1 = nn.Dropout(0.2)
        self.dense3_1 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))

        # Dense 2
        self.batch_norm3_2 = nn.BatchNorm1d(cha_po_2)
        self.dropout3_2 = nn.Dropout(0.2)
        self.dense3_2 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))

        # Dense 3
        self.batch_norm3_3 = nn.BatchNorm1d(cha_po_2)
        self.dropout3_3 = nn.Dropout(0.2)
        self.dense3_3 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))

    def forward(self, x):

        # Dense
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.celu(self.dense1(x), alpha=0.06)

        # Reshape
        x = x.reshape(x.shape[0],self.cha_1, self.cha_1_reshape)

        # CNN 1
        x = self.batch_norm_c1(x)
        x = self.dropout_c1(x)
        x = F.relu(self.conv1(x))

        # Avg-Pool
        x = self.ave_po_c1(x)

        # CNN 2_0
        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = F.relu(self.conv2(x))
        x_s = x

        # CNN 2_1
        x = self.batch_norm_c2_1(x)
        x = self.dropout_c2_1(x)
        x = F.relu(self.conv2_1(x))

        # CNN 2_2
        x = self.batch_norm_c2_2(x)
        x = self.dropout_c2_2(x)
        x = F.relu(self.conv2_2(x))
        x =  x * x_s

        # Max-Pool
        x = self.max_po_c2(x)

        # Flatten
        x = self.flt(x)

        # Dense 1
        x_1 = self.batch_norm3_1(x)
        x_1 = self.dropout3_1(x_1)
        x_1 = self.dense3_1(x_1)

        # Dense 2
        x_2 = self.batch_norm3_2(x)
        x_2 = self.dropout3_2(x_2)
        x_2 = self.dense3_2(x_2)

        # Dense 3
        x_3 = self.batch_norm3_3(x)
        x_3 = self.dropout3_3(x_3)
        x_3 = self.dense3_3(x_3)

        return x_1, x_2, x_3

In [103]:
# 모델 생성하기
model = CNN1D(num_features=NUM_FEATURES, hidden_size=HIDDEN_SIZE).to(device)

In [104]:
WEIGHT_DECAY = 0.00001

# Set optimizer, scheduler, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))

# Set loss functions for multihead classifier
loss_fn_1 = nn.CrossEntropyLoss()
loss_fn_2 = nn.CrossEntropyLoss()
loss_fn_3 = nn.CrossEntropyLoss()
loss_fn = [loss_fn_1, loss_fn_2, loss_fn_3]

# Set metrics
metric_fn = f1_score

In [105]:
# epoch별 절차를 정의하는 Trainer class

class CustomTrainer():

    """ Trainer
        epoch에 대한 학습 및 검증 절차 정의
    
    Attributes:
        model (`model`)
        device (str)
        loss_fn (Callable)
        metric_fn (Callable)
        optimizer (`optimizer`)
        scheduler (`scheduler`)
    """

    def __init__(self, model, device, loss_fn, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.model = model
        self.device = device

        self.loss_fn_1 = loss_fn[0]
        self.loss_fn_2 = loss_fn[1]
        self.loss_fn_3 = loss_fn[2]

        self.metric_fn = metric_fn
        self.optimizer = optimizer
        self.scheduler = scheduler

        # History - loss
        self.train_batch_loss_mean_list = list()
        self.train_batch_score_list = list()

        self.validation_batch_loss_mean_list = list()
        self.validation_batch_score_list = list()

        # History - predict
        self.train_target_pred_1_list = list()
        self.train_target_1_list = list()
        self.train_target_pred_2_list = list()
        self.train_target_2_list = list()
        self.train_target_pred_3_list = list()
        self.train_target_3_list = list()

        self.validation_target_pred_1_list = list()
        self.validation_target_1_list = list()
        self.validation_target_pred_2_list = list()
        self.validation_target_2_list = list()
        self.validation_target_pred_3_list = list()
        self.validation_target_3_list = list()

        # Output
        self.train_loss_mean = 0
        self.train_loss_sum = 0
        self.train_score = 0

        self.validation_loss_mean = 0
        self.validation_loss_sum = 0
        self.validation_score = 0


    def train_epoch(self, dataloader, epoch_index=0, verbose=False):
        """ 한 epoch에서 수행되는 학습 절차

        Args:
            dataloader (`dataloader`)
            epoch_index (int)
        """
        self.model.train()

        for batch_index, (feature, target_1, target_2, target_3) in enumerate(dataloader):
            feature, target_1, target_2, target_3 = feature.to(self.device), target_1.to(self.device), target_2.to(self.device), target_3.to(self.device)

            ## Pytorch에서는 gradients값들을 추후에 backward를 해줄때 계속 더해주기 때문에 
            ## 항상 backpropagation을 하기전에 gradients를 zero로 만들어주고 시작을 해야합니다.
            self.optimizer.zero_grad()

            # Summing up losses for Multihead
            target_pred_1, target_pred_2, target_pred_3 = self.model(feature)
            batch_loss_1_mean = self.loss_fn_1(target_pred_1, target_1)
            batch_loss_2_mean = self.loss_fn_2(target_pred_2, target_2)
            batch_loss_3_mean = self.loss_fn_3(target_pred_3, target_3)
            batch_loss_mean = batch_loss_1_mean + batch_loss_2_mean + batch_loss_3_mean
            batch_loss_sum = batch_loss_mean.item() * dataloader.batch_size
            self.train_batch_loss_mean_list.append(batch_loss_mean.item())
            self.train_loss_sum += batch_loss_sum

            # Calculate multiple batch scores for Multihead
            target_1_list = target_1.cpu().tolist()
            target_2_list = target_2.cpu().tolist()
            target_3_list = target_3.cpu().tolist()
            target_pred_1_list = torch.argmax(target_pred_1, dim=1).cpu().tolist()
            target_pred_2_list = torch.argmax(target_pred_2, dim=1).cpu().tolist()
            target_pred_3_list = torch.argmax(target_pred_3, dim=1).cpu().tolist()
            batch_score_1 = self.metric_fn(target_1_list, target_pred_1_list, average='macro')
            batch_score_2 = self.metric_fn(target_2_list, target_pred_2_list, average='macro')
            batch_score_3 = self.metric_fn(target_3_list, target_pred_3_list, average='macro')
            self.train_batch_score_list.append(sum([batch_score_1, batch_score_2, batch_score_3]) / 3)

            # History - predict
            self.train_target_1_list.extend(target_1_list)
            self.train_target_pred_1_list.extend(target_pred_1_list)
            self.train_target_2_list.extend(target_2_list)
            self.train_target_pred_2_list.extend(target_pred_2_list)
            self.train_target_3_list.extend(target_3_list)
            self.train_target_pred_3_list.extend(target_pred_3_list)

            batch_loss_mean.backward()
            self.optimizer.step()
            self.scheduler.step()


        self.train_loss_mean = self.train_loss_sum / len(dataloader)

        train_score_1 = self.metric_fn(self.train_target_1_list, self.train_target_pred_1_list, average='macro')
        train_score_2 = self.metric_fn(self.train_target_2_list, self.train_target_pred_2_list, average='macro')
        train_score_3 = self.metric_fn(self.train_target_3_list, self.train_target_pred_3_list, average='macro')
        self.train_score = sum([train_score_1, train_score_2, train_score_3]) / 3

        msg = f'Epoch {epoch_index}, Train, Mean loss: {self.train_loss_mean}, Score: {self.train_score}'


    def validate_epoch(self, dataloader, epoch_index=0):
        """ 한 epoch에서 수행되는 검증 절차

        Args:
            dataloader (`dataloader`)
            epoch_index (int)
        """
        self.model.eval()

        with torch.no_grad():
            for batch_index, (feature, target_1, target_2, target_3) in enumerate(dataloader):
                feature, target_1, target_2, target_3 = feature.to(self.device), target_1.to(self.device), target_2.to(self.device), target_3.to(self.device)

                self.optimizer.zero_grad()

                # Summing up losses for Multihead
                target_pred_1, target_pred_2, target_pred_3 = self.model(feature)
                batch_loss_1_mean = self.loss_fn_1(target_pred_1, target_1)
                batch_loss_2_mean = self.loss_fn_2(target_pred_2, target_2)
                batch_loss_3_mean = self.loss_fn_3(target_pred_3, target_3)
                batch_loss_mean = batch_loss_1_mean + batch_loss_2_mean + batch_loss_3_mean
                batch_loss_sum = batch_loss_mean.item() * dataloader.batch_size
                self.validation_batch_loss_mean_list.append(batch_loss_mean.item())
                self.validation_loss_sum += batch_loss_sum

                # Calculate multiple batch scores for Multihead
                target_1_list = target_1.cpu().tolist()
                target_2_list = target_2.cpu().tolist()
                target_3_list = target_3.cpu().tolist()
                target_pred_1_list = torch.argmax(target_pred_1, dim=1).cpu().tolist()
                target_pred_2_list = torch.argmax(target_pred_2, dim=1).cpu().tolist()
                target_pred_3_list = torch.argmax(target_pred_3, dim=1).cpu().tolist()
                batch_score_1 = self.metric_fn(target_1_list, target_pred_1_list, average='macro')
                batch_score_2 = self.metric_fn(target_2_list, target_pred_2_list, average='macro')
                batch_score_3 = self.metric_fn(target_3_list, target_pred_3_list, average='macro')
                self.validation_batch_score_list.append(sum([batch_score_1, batch_score_2, batch_score_3]) / 3)

                # History - predict
                self.validation_target_1_list.extend(target_1_list)
                self.validation_target_pred_1_list.extend(target_pred_1_list)
                self.validation_target_2_list.extend(target_2_list)
                self.validation_target_pred_2_list.extend(target_pred_2_list)
                self.validation_target_3_list.extend(target_3_list)
                self.validation_target_pred_3_list.extend(target_pred_3_list)

            self.validation_loss_mean = self.validation_loss_sum / len(dataloader)

            validation_score_1 = self.metric_fn(self.validation_target_1_list, self.validation_target_pred_1_list, average='macro')
            validation_score_2 = self.metric_fn(self.validation_target_2_list, self.validation_target_pred_2_list, average='macro')
            validation_score_3 = self.metric_fn(self.validation_target_3_list, self.validation_target_pred_3_list, average='macro')
            self.validation_score = sum([validation_score_1, validation_score_2, validation_score_3]) / 3

            msg = f'Epoch {epoch_index}, Validation, Mean loss: {self.validation_loss_mean}, Score: {self.validation_score}'

    
    def clear_history(self):
        """ 한 epoch 종료 후 history 초기화
            Examples:
                >>for epoch_index in tqdm(range(EPOCH)):
                >>    trainer.train_epoch(dataloader=train_dataloader, epoch_index=epoch_index, verbose=False)
                >>    trainer.validate_epoch(dataloader=validation_dataloader, epoch_index=epoch_index, verbose=False)
                >>    trainer.clear_history()
        """

        # History - loss
        self.train_batch_loss_mean_list = list()
        self.train_batch_score_list = list()

        self.validation_batch_loss_mean_list = list()
        self.validation_batch_score_list = list()

        # History - predict
        self.train_target_pred_1_list = list()
        self.train_target_1_list = list()
        self.train_target_pred_2_list = list()
        self.train_target_2_list = list()
        self.train_target_pred_3_list = list()
        self.train_target_3_list = list()

        self.validation_target_pred_1_list = list()
        self.validation_target_1_list = list()
        self.validation_target_pred_2_list = list()
        self.validation_target_2_list = list()
        self.validation_target_pred_3_list = list()
        self.validation_target_3_list = list()

        # Output
        self.train_loss_mean = 0
        self.train_loss_sum = 0
        self.train_score = 0

        self.validation_loss_mean = 0
        self.validation_loss_sum = 0
        self.validation_score = 0


In [106]:

class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        """ 초기화

        Args:
            patience (int): loss가 줄어들지 않아도 학습할 epoch 수
            weight_path (str): weight 저장경로
            verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        """
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단

        Args:
            loss (float):

        Examples:
            
        Note:
            
        """  

        if self.min_loss == np.Inf:
            #첫 에폭
            self.min_loss = loss
            # self.save_checkpoint(loss=loss, model=model)

        elif loss > self.min_loss:
            # loss가 줄지 않음 -> patience_counter 1 증가
            self.patience_counter += 1
            msg = f"Early stopper, Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True


                
        elif loss <= self.min_loss:
            # loss가 줄어듬 -> min_loss 갱신
            self.save_model = True
            msg = f"Early stopper, Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss


In [107]:
# Trainer 셋팅하기
trainer = CustomTrainer(model, device, loss_fn, metric_fn, optimizer, scheduler)

# Earlystopper 셋팅하기
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

## 학습
각 에폭별 수행할 절차를 trainer 클래스의 train_epoch, validation_epoch을 이용해 정의합니다.  
EPOCHS만큼, 혹은 early_stop이 될 때까지 에폭을 반복합니다.

In [79]:
import time

MODEL_DIR = os.path.join(ROOT_PATH, 'best.pt')
start = time.time()

criterion = 0

# 에폭 별로 train_epoch, valid_epoch 실헹힘
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index=epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index=epoch_index)
   
    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.validation_loss_mean)
        
    if early_stopper.stop:
        print('Early stopped')
        break
    
    
    if trainer.validation_score > criterion:
        # 모델이 개선됨 -> 검증 점수와 weight 갱신
        criterion = trainer.validation_score
        torch.save(model.state_dict(), MODEL_DIR)
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, os.path.join(ROOT_PATH, 'best.pt'))


    trainer.clear_history()

print("Model saved: best.pt")

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

## 추론
테스트 데이터의 타겟 변수를 `sample_submission.csv` 양식에 맞춰 저장한 파일을 플랫폼을 통해 제출하면 추론 점수를 확인할 수 있습니다.  

"역률", "전류고조파", "전압고조파" 컬럼 값을 여러분의 모델의 추론 결과로 채워 제출 파일을 만듭니다 (현재는 모두 아래 보시는 바와 같이 동일한 값으로 채워져 있습니다). INDEX 값을 기준으로 채점을 진행하는 점 유의해주시기 바랍니다.

In [51]:
submit = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
submit.head()

,index,누적전력량,유효전력평균,무효전력평균,주파수,전류평균,상전압평균,선간전압평균,온도,R상유효전력,...,S상전압,S상선간전압,T상유효전력,T상무효전력,T상전류,T상전압,T상선간전압,label_역률평균,label_전류고조파평균,label_전압고조파평균
0,0,2248930.5,28963.0,20237.0,59.854076,45.197918,259.916656,449.916656,47.500,8663.0,...,260.75,452.75,10417.0,7304.0,48.71875,260.75,448.75,정상,경고,주의
1,1,0.0,0.0,0.0,0.000000,101.312500,0.000000,0.000000,24.375,0.0,...,0.00,0.00,0.0,0.0,100.68750,0.00,0.00,경고,정상,정상
2,2,5375707.0,35244.0,19826.0,59.975650,107.385414,125.416664,216.750000,18.750,11988.0,...,125.25,216.25,12236.0,6170.0,109.56250,125.00,216.25,정상,정상,주의
3,3,17781200.0,77056.0,39520.0,59.863000,244.854000,118.083000,205.333000,23.125,25796.0,...,118.75,118.75,24992.0,13704.0,242.18800,118.00,118.00,정상,정상,경고
4,4,10143988.0,0.0,0.0,59.798140,0.000000,133.750000,231.500000,26.875,0.0,...,134.50,231.50,0.0,0.0,0.00000,133.00,230.25,경고,정상,정상


In [52]:
# 테스트 데이터 로드
test_dataset = CustomDataset(data_dir=DATA_DIR, mode='test')
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


In [53]:
len(test_dataset)

313267

In [55]:
""" 이전에 학습한 모델 weight파일을 불러 추론하려면 아래 주석을 풀고 실행
    학습 진행 후 바로 추론하는 경우 학습 과정의 model 사용 (주석 풀지 않고 실행) """

MODEL_DIR = os.path.join(ROOT_PATH, 'best.pt')

# 모델 사이즈와 맞춰주기 위해 재설정
HIDDEN_SIZE = 4096

model = CNN1D(NUM_FEATURES, hidden_size=HIDDEN_SIZE)
model.load_state_dict(torch.load(MODEL_DIR)['model'])

# 추론
model.eval()

# 추론 결과를 pred 리스트로 저장
pred1 = []
pred2 = []
pred3 = []
with torch.no_grad():
    ##train_epoch 참고
    for batch_index, (feature) in enumerate(test_dataloader):
        output1, output2, output3 = model(feature)

        pred1.extend(torch.argmax(output1, dim=1).cpu().tolist())
        pred2.extend(torch.argmax(output2, dim=1).cpu().tolist())
        pred3.extend(torch.argmax(output3, dim=1).cpu().tolist())
       
        # 진행과정 출력
        if batch_index % 50 == 0:
            print(f'Prediction: {batch_index}/{len(test_dataloader)} completed')
    print("Prediction all completed")
    

FileNotFoundError: [Errno 2] No such file or directory: 'best.pt'

In [ ]:
print(len(pred1))

In [57]:

# 인코딩 값으로 나온 타겟 변수를 디코딩
pred1 = test_dataset.label_decoder(pred1)
pred2 = test_dataset.label_decoder(pred2)
pred3 = test_dataset.label_decoder(pred3)

print('decode COMPLETED')


NameError: name 'pred1' is not defined

In [ ]:
# 제출 파일 확인
submit['label_역률평균'] = pred1
submit['label_전류고조파평균'] = pred2
submit['label_전압고조파평균'] = pred3
submit.head()

In [ ]:
# 제출 파일 제작
submit.to_csv(os.path.join(ROOT_PATH,'submission.csv'), index=False)